# An idea
Make a database of recipes with tags for nutrition info, vegetaruan, vegan, etc. Then let users enter their dietary requirements - calories per day, macros, vegan, do you snack, etc - and generate a weekly meal plan to meet those requirements.

Upgrades could include uploading current weight, goal weight and date, and automatically calculating calories per day. Switching any item in the menu for another one that meets the requirements. A recommender system that sees which recipes you normally like (you can rate them) and suggests similar ones. A slider bar that adjusts what fraction of your suggested meals are similar to others and which ones are new suggestions.

Ideally, when a user initiates their account, they enter current weight, goal weight, goal date, nutritional needs, and rate with stars a few representative meals so that we get an idea of what sort of recipes they prefer.

Eventually, if this were to work with epicure or something like that, it would generate an automatic shopping list with links to their products.

It would probably be best to use SQL to store recipes, for scalability.

I am using https://rogerbinns.github.io/apsw/download.html plus DB browser for SQL lite (desktop app).

A tool I would personally like it a thing for my phone where it suggests what I should eat for the day, but if I eat something different, I can enter it, and then it will re-adjust with suggestions for the rest of the day. Also, it should remember what I have eaten in the past so I can see insights and maybe it can learn to suggest things I like.

In [2]:
import apsw  # that was a bitch to install; only works with python 3 on my machine
import numpy as np

In [3]:
# Opening/creating database
dbfile = "recipes_db/recipes.db"
connection = apsw.Connection(dbfile)
cursor = connection.cursor()

### TODO: what is a connection and what is a cursor?


Here is an example of how to use apsw: https://rogerbinns.github.io/apsw/example.html

In [4]:
# for now, start with selecting some data from my db
for row in cursor.execute("select * from recipes"):
    print(row)

(1, '100 calorie greek yogurt', 'greek', 100, 1, 1, None, 1)
(2, 'Pan-fried potatoes', 'american', '', 1, 1, 1, '')


Cool, that's the basic functionality I need. Now let's make an algorithm that generates a daily menu.
# Make a function that will choose a meal
Given a dictionary of nutrition requirements, a list of tolerances on those nutrition requirements, and the type of meal we want.

In [14]:
# template inputs for query
# goal nutrition intake for this meal
nutrition_reqs = {'calories': 100,
                  'carbs': None ,
                  'fat': None,
                  'protein': None ,
                  'sodium': None 
                 }
# tuples with upper and lower tolerances
nutrition_tol = {'calories': (),
                 'carbs': (),
                 'fat': (),
                 'protein': (),
                 'sodium': (), 
                }

In [16]:
def choose_meal(nutrition_reqs, nutrition_tol, meal_type):
    '''Choose a suitable meal from the database.
    
    Inputs:
    nutrition_reqs: a dict of nutrition requirements for this meal
    nutrition_tol: a dict of nutrition tolerance for this meal 
    c
    
    Output: 
    meal: an object corresponding to the selected row of the recipes db
    '''
    # query the database for all meals that meet the criteria 
    query_out = cursor.execute("""SELECT * FROM recipes 
                               WHERE
                               (calories BETWEEN {} AND {})
                               AND (carbs BETWEEN {} AND {})
                               AND (fat BETWEEN {} AND {})
                               AND (protein BETWEEN {} AND {})
                               AND (sodium BEWEEN {} AND {})
                               AND (meal_type == {})
                               """.format(nutrition_reqs['calories'], nutrition_tol['calories'][0],  nutrition_tol['calories'][1],
                                          nutrition_reqs['carbs'], nutrition_tol['carbs'][0],  nutrition_tol['carbs'][1],
                                          nutrition_reqs['fat'], nutrition_tol['fat'][0],  nutrition_tol['fat'][1],
                                          nutrition_reqs['protein'], nutrition_tol['protein'][0],  nutrition_tol['protein'][1],
                                          nutrition_reqs['sodium'], nutrition_tol['sodium'][0],  nutrition_tol['sodium'][1],
                                          meal_type
                                         ))
    # choose a meal at random
    # TODO: later, a more sophisticated version of this could be done
    # choose in some non-random way
    meal = query_out[np.random.uniform(0, len(query_out))]
    
    return(meal)

## Make a function that will make a meal plan for an entire day 

Here's a question- how do I set the nutrition requirements and tolerances for each meal? What is the optimum way to divide nutrients throughout the meals in a day? Maybe I should start by defining the relative size of meals I prefer. I'll give each a number from 0 to 1 indicating how much of my daily nutrients I like to have with each meal...

Dinner is important to me; 0.3 seems reasonable. For lunch, I like a decent amount as well, so let's give lunch 0.25. Breakfast should have 0.2. This leaves 0.25 for snacks and dessert, so I'll just split it in two and give them each 0.125. I'll call these vaules `meal_significance`.

In [27]:
meal_significance = {'breakfast': 0.2,
                    'lunch': 0.25,
                    'dinner': 0.3,
                    'snack': 0.125,
                    'dessert': 0.125
                    }
assert(np.sum(list(meal_significance.values())) == 1.)

In [28]:
def get_meal_nutrition(nutrition_reqs_daily, nutrition_tol_daily, meal_significance, meal_type):
    '''Define the goal nutrition requirements and tolerance for a given meal. 
    
    Inputs:
    nutrition_reqs_daily: a dict of nutrition requirements for this day
    nutrition_tol_daily: a dict of nutrition tolerance for this day 
    meal_significance: a dict of relative number of goal nutrients per meal
    meal_type: a string, can be "breakfast", "lunch", "dinner", "snack", or "dessert"
    
    Outputs:
    nutrition_reqs_meal: a dict of nutrition requirements for this meal
    nutrition_tol_meal: a dict of nutrition tolerance for this meal 
    '''
    nutrition_reqs_meal = {'calories': nutrition_reqs_daily['calories'] * meal_significance[meal_type],
                           'carbs': nutrition_reqs_daily['carbs'] * meal_significance[meal_type] ,
                           'fat': nutrition_reqs_daily['fat'] * meal_significance[meal_type],
                           'protein': nutrition_reqs_daily['protein'] * meal_significance[meal_type] ,
                           'sodium': nutrition_reqs_daily['sodium'] * meal_significance[meal_type] 
                          }
    # tuples with upper and lower tolerances
    nutrition_tol_meal = {'calories': (nutrition_tol_daily['calories'][0] * meal_significance[meal_type],
                                       nutrition_tol_daily['calories'][1] * meal_significance[meal_type]),
                          'carbs': (nutrition_tol_daily['carbs'][0] * meal_significance[meal_type],
                                    nutrition_tol_daily['carbs'][1] * meal_significance[meal_type]),
                          'fat': (nutrition_tol_daily['fat'][0] * meal_significance[meal_type],
                                  nutrition_tol_daily['fat'][1] * meal_significance[meal_type]),
                          'protein': (nutrition_tol_daily['protein'][0] * meal_significance[meal_type],
                                      nutrition_tol_daily['protein'][1] * meal_significance[meal_type]),
                          'sodium': (nutrition_tol_daily['sodium'][0] * meal_significance[meal_type],
                                    nutrition_tol_daily['sodium'][1] * meal_significance[meal_type])
                         }
    
    return(nutrition_reqs_meal, nutrition_tol_meal)

TODO: need to add functionality that adjusts nutrition requirements and tolerances for other meals in a day if one meal is particularly light or heavy.

In [ ]:
def make_daily_plan(nutrition_reqs_daily, nutrition_tol_daily, meal_significance):
    '''Choose a suitable set of meals for the day from the database.
    
    Inputs:
    nutrition_reqs_daily: a dict of nutrition requirements for this day
    nutrition_tol_daily: a dict of nutrition tolerance for this day 
    meal_significance: a dict of relative number of goal nutrients per meal
    
    
    Outputs:
    breakfast, lunch, dinner, snack, dessert: objects corresponding to the selected
                                              row of the recipes db for each meal
    '''
    breakfast = choose_meal(br_nutrition_reqs, br_nutrition_tol, meal_type='breakfast')
    lunch = choose_meal(lu_nutrition_reqs, lu_nutrition_tol, meal_type='lunch')
    dinner = choose_meal(di_nutrition_reqs, di_nutrition_tol, meal_type='dinner')
    snack = choose_meal(sn_nutrition_reqs, sn_nutrition_tol, meal_type='snack')
    dessert = choose_meal(de_nutrition_reqs, de_nutrition_tol, meal_type='dessert')
    
    return(breakfast, lunch, dinner, snack, dessert)